In [1]:
import numpy as np 
from game import Game, Player, Move, Board
from copy import deepcopy
from tqdm.notebook import tqdm
from time import time

In [2]:
g = Game()
g.get_board()

array([[-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]], dtype=int8)

In [3]:
def score(player, state : Board) :
    board = state.board
    score = 5
    for row in board :
        new_score = np.count_nonzero(row == player)
        score = min(score, 5 - new_score)
    
    for col in range(board.shape[1]) :
        new_score = np.count_nonzero(board[:,col] == player)
        score = min(score, 5 - new_score)
    
    diag1 = board.diagonal()
    new_score = np.count_nonzero(diag1 == player)
    score = min(score, 5 - new_score)

    diag2 = np.fliplr(board).diagonal()
    new_score = np.count_nonzero(diag2 == player)
    score = min(score, 5 - new_score)

    return score

In [4]:
def get_periphery_board(board) :
    shape_y, shape_x = board.shape
    periphery_cubes = set()
    for i in range(shape_x) :
        periphery_cubes.add((0,i))
        periphery_cubes.add((shape_y-1,i))
    for j in range(shape_y) :
        periphery_cubes.add((j,0))
        periphery_cubes.add((j, shape_x-1))
    
    return periphery_cubes

In [5]:
def get_successors(player, board : Board) :
    '''Check which cube can be played'''
    playable_cubes = []
    for pos in get_periphery_board(board) :
        if board[pos] == -1 or board[pos] == player :
            playable_cubes.append(pos)
    
    '''For each playable cube, compute the possible successors, depending on the movement of slide, and returns the move and the associated board'''
    successors = []
    for pos in playable_cubes :
        for slide_direction in board.acceptable_slides(pos) :
            new_board = deepcopy(board)
            new_board[pos] = player
            new_board.slide(pos, slide_direction)
            # Coordinates are inverted in the Game class, so they are returned inverted here for conversion
            successors.append([((pos[1], pos[0]), slide_direction), new_board.board])
        
    return successors
    

In [9]:
class Tree :
    def __init__(self, board = None, children : list = None) -> None:
        self.board = board
        self.children = children if children is not None else dict()
        # Moves are the plays corresponding one to one to the children boards
        self.score = -1        
    
    def get_leaves(self) :
        if self.children == dict() :
            return [self]
        leaves = []
        for move, node in self.children.items() :
            leaves.extend(node.get_leaves())
        
        return leaves
    
'''Computes all the next possible moves and boards until a given depth is reached, for further application of MinMax'''
def get_states_tree(player : int, board : Board, max_time : int, initial_player : int,  max_depth = 1, start_time = None, current_depth = 0) :
    if start_time is None :
        start_time = time()
        
    tree = Tree(board)
    if board.check_winner() != -1 or (time() - start_time) > max_time or current_depth >= max_depth :
        return tree

    for move, succ in get_successors(player, board) :
        tree.children[move] = get_states_tree(1 - player, Board(succ), max_time, initial_player, max_depth, start_time, current_depth + 1)

    return tree

'''Computes the score of the leaves of the tree (furthest anticipated moves) for further application of MinMax'''
def valuate_tree(player : int, states_tree : Tree, score_function = score) :
    valuated_tree = deepcopy(states_tree)
    for leaf in valuated_tree.get_leaves() :
        leaf.score = score(player, leaf.board)
    
    return valuated_tree

In [12]:
g = Game()
t = get_states_tree(0, Board(g.get_board()), 30, 0, 4)

In [36]:
list(list(t.children.values())[7].children.values())[40]

In [7]:
'''MinMax Algorithm where the root node is always the player who tries to maximize the score --> Returns the Move as well as the score, as we want to know what to play'''
def min_max(valuated_tree : Tree, compute_max=False) :
    if valuated_tree.children == dict() :
        return None, valuated_tree.score
    
    options = []
    for move, child in valuated_tree.children.items() :
        options.append([move, min_max(child, compute_max=bool(1-compute_max))[1]])
    
    if compute_max :
        return max(options, key = lambda t : t[1])
    else : 
        return min(options, key = lambda t : t[1])
    

In [8]:
class MinMaxPlayer(Player) :
    def __init__(self, score_function = score) -> None:
        super().__init__()
        self.score_function = score_function
    
    def make_move(self, game: Game) -> tuple[tuple[int, int], Move]:
        board = Board(game.get_board())
        player = game.get_current_player()
        tree = get_states_tree(player, board, 2, player)
        value_tree = valuate_tree(player, tree, self.score_function)
        move, score = min_max(value_tree)
        return move
        

In [9]:
from main import RandomPlayer

g = Game()
g.play(RandomPlayer(), MinMaxPlayer())
g.get_board()

array([[-1,  1, -1, -1,  0],
       [ 0,  1, -1, -1, -1],
       [-1,  1, -1, -1, -1],
       [-1,  1,  0, -1, -1],
       [-1,  1,  0, -1, -1]], dtype=int8)

In [10]:
'''Simulates a certain number of games between two strategies, and displays the final performance of both strategies. To avoid potential edge associated to being the beginning player, player1 plays half the time first, half the time second.'''
def evaluate_strategies(player1 : Player, player2 : Player, deterministic_only = False) :
    # If at least one strategy includes randomness, we run 100 games to get reliable results
    if not deterministic_only :
        number_of_games = 100
    # If both strategies are deterministic, then running several games won't change anything in the results (we only need to play 2 games, one for each starting position)
    else :
        number_of_games = 2
    half_games = number_of_games // 2
    victories_1 = victories_2 = 0
    for _ in tqdm(range(half_games)) :
        g = Game()
        g.play(player1, player2)
        if g.check_winner() == 0 :
            victories_1 += 1
        else : 
            victories_2 += 1
    
    for _ in tqdm(range(half_games)) :
        g = Game()
        g.play(player2, player1)
        if g.check_winner() == 1 :
            victories_1 += 1
        else :
            victories_2 += 1
    
    return victories_1, victories_2

In [11]:
evaluate_strategies(MinMaxPlayer(), RandomPlayer())

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

(92, 8)

In [12]:
board = Board(np.array([[ 0, -1, -1, -1, -1],
 [ 0, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [ 1, -1, -1, -1, -1]]))

t = get_states_tree(1, board, 2, 1)

In [13]:
v = valuate_tree(1, t, score)

In [14]:
m = min_max(v)

In [23]:
time()

1705712649.1574888

## TODO :
- ~~In get_successors, return the Moves associated to the next boards~~
- ~~Store the Move associated to the Board in the Tree (In the form of a Dict[move : child])~~
- ~~Change MinMax to work on dict[child : score of MinMax] to be able to return a direct child of the root and not a leaf of the tree~~
- ~~Optimize trees generation to have quicker games (Try caching and/or combine tree generation and valuation to have Alpha-Beta pruning during generation)~~
- Change the generation of trees to be in breadth-first, with only time-limit, no more depth limit
- Improve score function to be more performant
- ~~See if it is possible to adapt the max_depth of the Tree based on the device capabilities~~
- Add the possibility to play against AI

In [16]:
g = Game()
t = get_states_tree(0, Board(g.get_board()), 2, 0)
v = valuate_tree(0, t)
min_max(v, True)

[((1, 0), <Move.BOTTOM: 1>), 4]

In [17]:
board = np.array([[ 0, 0, 0, 1,-1],
                  [ 1,-1,-1,-1, 0],
                  [ 0, 1, 1,-1,-1],
                  [ 1, 0,-1,-1,-1],
                  [ 0, 1, 1, 0,-1]])
t = get_states_tree(0, Board(board), 3, 0)
v = valuate_tree(0, t)
min_max(v, compute_max=False)

[((1, 0), <Move.BOTTOM: 1>), 1]

In [18]:
s = get_states_tree(0, Board(board), 3, 0)
v = valuate_tree(0,s)
for succ in v.children.items() :
    print(succ[1].score)

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [19]:
max([1,2,3])

3